The goal of this notebook is to provide step by step explanation of the internal workings of PySyft for developers and have working examples of the API to play with.

In [8]:
import sys
import pytest
import syft as sy
from syft.core.node.common.service.auth import AuthorizationException
from syft.util import key_emoji
sy.LOG_FILE = "syft_do.log"
sy.logger.remove()
_ = sy.logger.add(sys.stdout, level="DEBUG")

Bob has some data he wants to share with Alice using PySyft. The first thing Bob needs to do is to create a Node to handle all the PySyft services he will need to safely and securely share his data with Alice.

In [9]:
some_device = sy.Device()

[2021-09-06T12:42:00.838782+0000][DEBUG][logger]][17408] > Creating 🗃️ (MemoryStore) {}
[2021-09-06T12:42:00.839754+0000][DEBUG][logger]][17408] Created MemoryStore.
[2021-09-06T12:42:00.840795+0000][DEBUG][logger]][17408] > Creating 📱 Festive Goldberg (Device)@<UID:🚸🚂>


This is a device, it is a Node and it has a name, id and address.

In [10]:
print(f'Device name: {some_device.name}')
print(f'Device id: {some_device.id}')
print(f'Device adress: {some_device.address}')

Device name: Festive Goldberg
Device id: <UID: 3176a99e4a1f4d35b02dcdaba6459c5e>
Device adress: <Address - Device:<SpecificLocation:..59c5e>>


The ID is a class called UID which is essentially a UUID (universally unique identifier). 
The address is a combination of up to four different locations, identifying the path to resolve the final target of the address.

In [13]:
print(some_device.address.vm) 
print(some_device.address.device)
print(some_device.address.domain)
print(some_device.address.network)
print(some_device.address.target_id)

None
<SpecificLocation: 3176a99e4a1f4d35b02dcdaba6459c5e>
None
None
<SpecificLocation: 3176a99e4a1f4d35b02dcdaba6459c5e>


UIDs are hard to read and compare so we have a function which converts them to 2 x emoji. 
Just like the "name", Emoji uniqueness is not guaranteed but is very useful during debugging.

In [15]:
print(some_device.address.target_id.pprint)
print(some_device.address.pprint)

📌 Festive Goldberg (SpecificLocation)@<UID:🚸🚂>
💠 [📱] Festive Goldberg (Address)


In [19]:
print(some_device.id, some_device.address.target_id.id)
print(some_device.id.emoji(), "==", some_device.address.target_id.id.emoji())
print(some_device.pprint)
assert some_device.id == some_device.address.target_id.id

<UID: 3176a99e4a1f4d35b02dcdaba6459c5e> <UID: 3176a99e4a1f4d35b02dcdaba6459c5e>
<UID:🚸🚂> == <UID:🚸🚂>
📱 Festive Goldberg (Device)@<UID:🚸🚂>


Interaction with a Node like a device is always done through a client. 
Clients can "send" messages and Nodes can "receive" them. 
Bob needs to get a client for his device. 
But first it might be a good idea to name the device so that it's easier to follow.

In [20]:
bob_device = sy.Device(name="Bob's iPhone")
assert bob_device.name == "Bob's iPhone"

[2021-09-06T12:56:30.611050+0000][DEBUG][logger]][17408] > Creating 🗃️ (MemoryStore) {}
[2021-09-06T12:56:30.611877+0000][DEBUG][logger]][17408] Created MemoryStore.
[2021-09-06T12:56:30.612957+0000][DEBUG][logger]][17408] > Creating 📱 Bob's iPhone (Device)@<UID:🚩🙾>


In [21]:
bob_device_client = bob_device.get_client()

[2021-09-06T12:56:42.644787+0000][INFO][logger]][17408] Skipping torch.Tensor.fft not supported in 1.8.1
[2021-09-06T12:56:42.645547+0000][INFO][logger]][17408] Skipping torch.Tensor.ifft not supported in 1.8.1
[2021-09-06T12:56:42.646056+0000][INFO][logger]][17408] Skipping torch.Tensor.irfft not supported in 1.8.1
[2021-09-06T12:56:42.646523+0000][INFO][logger]][17408] Skipping torch.Tensor.rfft not supported in 1.8.1
[2021-09-06T12:56:42.651269+0000][INFO][logger]][17408] Skipping torch.ifft not supported in 1.8.1
[2021-09-06T12:56:42.651883+0000][INFO][logger]][17408] Skipping torch.irfft not supported in 1.8.1
[2021-09-06T12:56:42.652362+0000][INFO][logger]][17408] Skipping torch.rfft not supported in 1.8.1
[2021-09-06T12:56:42.782113+0000][DEBUG][logger]][17408] > Creating 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾>


When you ask a node for a client you get a "Client" which is named after the device and has 
the same "UID" and "Address" (4 x Locations) as the device it was created from, 
and it will have a "Route" that connects it to the "Device"

In [22]:
assert bob_device_client.name == "Bob's iPhone Client"

In [23]:
print(bob_device_client.pprint, bob_device.pprint)
print(bob_device.id.emoji(), "==", bob_device_client.id.emoji())
assert bob_device.id == bob_device_client.device.id
assert bob_device.address == bob_device_client.address

📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾> 📱 Bob's iPhone (Device)@<UID:🚩🙾>
<UID:🚩🙾> == <UID:🚩🙾>


Now we have something that can send and receive lets take it for a spin. 
Since everything is handled with a layer of abstraction the smallest unit of work is a 
"SyftMessage". Very little can be done without sending a message from a Client to a Node. 
There are many types of "SyftMessage" which boil down to whether or not they are Sync or Async, 
and whether or not they expect a response.

Lets make a ReprMessage which simply gets a message and prints it at its destination Node. 
SyftMessage's all have an "address" field, without this they would never get delivered. They also generally have a msg_id which can be used to keep track of them.

In [24]:
msg = sy.ReprMessage(address=bob_device_client.address)
print(msg.pprint)
print(bob_device_client.address.pprint)
assert msg.address == bob_device_client.address

[2021-09-06T13:13:57.791742+0000][DEBUG][logger]][17408] > Creating ✉️  (ReprMessage) <UID:🙽🚄>
✉️  (ReprMessage)
💠 [📱] Bob's iPhone Client (Address)


In [25]:
print(sy.ReprMessage.mro())

[<class 'syft.core.node.common.service.repr_service.ReprMessage'>, <class 'syft.core.common.message.ImmediateSyftMessageWithoutReply'>, <class 'syft.core.common.message.ImmediateSyftMessage'>, <class 'syft.core.common.message.SyftMessageWithoutReply'>, <class 'syft.core.common.message.SyftMessage'>, <class 'syft.core.common.message.AbstractMessage'>, <class 'syft.core.common.object.ObjectWithID'>, <class 'syft.core.common.serde.serializable.Serializable'>, <class 'typing.Generic'>, <class 'object'>]


In [26]:
with pytest.raises(AuthorizationException):
    bob_device_client.send_immediate_msg_without_reply(
        msg=sy.ReprMessage(address=bob_device_client.address)
    )

[2021-09-06T13:16:57.612497+0000][DEBUG][logger]][17408] > Creating ✉️  (ReprMessage) <UID:🙉🜓>
[2021-09-06T13:16:57.613479+0000][DEBUG][logger]][17408] > 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾> Signing ✉️  (ReprMessage) with 🛃
[2021-09-06T13:16:57.614027+0000][DEBUG][logger]][17408] > Signing with 🛃
[2021-09-06T13:16:57.614495+0000][DEBUG][logger]][17408] Serializing <class 'syft.core.node.common.service.repr_service.ReprMessage'>
[2021-09-06T13:16:57.615893+0000][DEBUG][logger]][17408] > Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🙉🜓>
[2021-09-06T13:16:57.616603+0000][DEBUG][logger]][17408] > Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾> ➡️  💠 [📱] Bob's iPhone Client (Address)
[2021-09-06T13:16:57.617134+0000][DEBUG][logger]][17408] > Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
[2021-09-06T13:16:57.618205+0000][DEBUG][logger]][17408] > Creating ✉️  (ReprMessage) <UI

In [27]:
print(bob_device_client.keys)
print(bob_device.keys)

assert bob_device_client.verify_key != bob_device.root_verify_key

🔑 🛃
🔑 🞁🗝 🞁


In [28]:
bob_device_client = bob_device.get_root_client()

[2021-09-06T13:17:47.645528+0000][INFO][logger]][17408] Skipping torch.Tensor.fft not supported in 1.8.1
[2021-09-06T13:17:47.646486+0000][INFO][logger]][17408] Skipping torch.Tensor.ifft not supported in 1.8.1
[2021-09-06T13:17:47.646997+0000][INFO][logger]][17408] Skipping torch.Tensor.irfft not supported in 1.8.1
[2021-09-06T13:17:47.647867+0000][INFO][logger]][17408] Skipping torch.Tensor.rfft not supported in 1.8.1
[2021-09-06T13:17:47.652535+0000][INFO][logger]][17408] Skipping torch.ifft not supported in 1.8.1
[2021-09-06T13:17:47.653088+0000][INFO][logger]][17408] Skipping torch.irfft not supported in 1.8.1
[2021-09-06T13:17:47.653593+0000][INFO][logger]][17408] Skipping torch.rfft not supported in 1.8.1
[2021-09-06T13:17:47.956658+0000][DEBUG][logger]][17408] > Creating 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾>


In [29]:
bob_device_client.send_immediate_msg_without_reply(
    msg=sy.ReprMessage(address=bob_device_client.address)
)


[2021-09-06T13:19:12.613958+0000][DEBUG][logger]][17408] > Creating ✉️  (ReprMessage) <UID:🙽🚰>
[2021-09-06T13:19:12.614973+0000][DEBUG][logger]][17408] > 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾> Signing ✉️  (ReprMessage) with 🚡
[2021-09-06T13:19:12.615815+0000][DEBUG][logger]][17408] > Signing with 🚡
[2021-09-06T13:19:12.616360+0000][DEBUG][logger]][17408] Serializing <class 'syft.core.node.common.service.repr_service.ReprMessage'>
[2021-09-06T13:19:12.617635+0000][DEBUG][logger]][17408] > Creating Signed ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) <UID:🙽🚰>
[2021-09-06T13:19:12.618223+0000][DEBUG][logger]][17408] > Sending ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) 📡 [📱] Bob's iPhone Client (DeviceClient)@<UID:🚩🙾> ➡️  💠 [📱] Bob's iPhone Client (Address)
[2021-09-06T13:19:12.618690+0000][DEBUG][logger]][17408] > Routing ✉️ 🔏 (SignedImmediateSyftMessageWithoutReply) via 🛣️  (SoloRoute)
[2021-09-06T13:19:12.619340+0000][DEBUG][logger]][17408] > Creating ✉️  (ReprMessage) <UI